# Using Selenium in Chrome to scrape relevant charity data from charitydata.ca (collected from T3010s submitted to the CRA)

In [82]:
#importing required libraries and setting the driver path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


driver_path = "C:/Users/16475/Documents/GitHub/CharityScore.ca/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [83]:
#import the url data from the dataframe
df = pd.read_csv("data.csv")
df = pd.read_csv("data.csv").drop(['Unnamed: 0'],axis=1)

for i in range(1,2):
    driver.get(df.loc[i,"URL"])
    
    # open expense table 
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, 'Expenses'))).click()
    expenses_table = driver.find_element_by_xpath('//html/body/main/div/section[3]/div/div[2]/div[4]/div[2]/table[1]')
    print(expenses_table.get_attribute('innerHTML'))

    

<colgroup><col span="1" class="wide"></colgroup><thead><tr style="height: 32px;"><th scope="col">Expenditures by 6 Groups</th><th>2020</th><th>2019</th><th>2018</th><th>2017</th><th>2016</th><th>2015</th><th>2014</th><th>2013</th><th>2012</th><th>2011</th><th>2010</th><th>2009</th><th>2008</th><th>2007</th><th>2006</th><th>2005</th><th>2004</th><th>2003</th></tr></thead><tbody><tr style="height: 51px;"><th scope="row">(a) Total expenditures on charitable activities ($)</th><td class="num">75,367</td><td class="num">91,549</td><td class="num">80,577</td><td class="num">78,587</td><td class="num">78,654</td><td class="num">82,129</td><td class="num">96,162</td><td class="num">99,835</td><td class="num">0</td><td class="num">90,861</td><td class="num">62,563</td><td class="num">56,634</td><td class="num">65,860</td><td class="num">52,684</td><td class="num">28,982</td><td class="num">30,674</td><td class="num">0</td><td class="num">0</td></tr><tr style="height: 69px;"><th scope="row">(b) 